In [0]:
%sql
use catalog dev_edh;
use schema dummy;
--select count(*) from dev_edh.dummy.delta_source;
--truncate TABLE dev_poc.default.delta_source;


In [0]:
import pandas as pd
import warnings
import time
warnings.simplefilter(action='ignore', category=FutureWarning)
dbutils.widgets.text(label="Source Directory", name="source_path", defaultValue='')


In [0]:
%python
import os
import pandas as pd
import time

data = []
cols = ['FileName', 'SheetName', 'ModificationTime', 'LoadTime', 'Data']
directory = '/dbfs/user/arundhuti/delta/'
list_files = os.listdir(directory)

for item in list_files:
    file_path = os.path.join(directory, item)
    if item[-5:] == '.xlsx':
        sheets = pd.ExcelFile(file_path)
        if len(sheets.sheet_names) >= 1:
            for sheet in sheets.sheet_names:
                pdf = pd.read_excel(file_path, sheet_name=sheet)
                modfied_time = int(os.path.getmtime(file_path) / 1000)
                value = pdf.to_json(orient='split')
                load_time = time.strftime("%Y%m%d:%H:%M:%S", time.localtime())

                data.append([file_path, sheet, time.strftime('%Y%m%d:%H:%M:%S', time.localtime(modfied_time)), load_time, value])
                df = spark.createDataFrame(data, cols)
                df.write.format("delta").mode("append").saveAsTable("delta_source")

In [0]:
display(df)

In [0]:
df2 = df.filter((df['FileName'] == '/dbfs/user/arundhuti/delta/data_detail.xlsx') & (df['SheetName'] == 'Member')).select('Data')
display(df2)

In [0]:
import json

df3 = json.loads(df2.collect()[0]['Data'])
cols = df3['columns']
data = df3['data']
sdf = spark.createDataFrame(data, cols)
display(sdf)
#df3

In [0]:
current_time = time.strftime("%Y%m%d:%H:%M:%S",time.localtime())
current_time 


In [0]:
%sql
select * from dev_edh.dummy.delta_source